In [1]:
import pandas as pd
import numpy as np

In [2]:
def read_files(file, method, class_):
    with open(file, 'r') as f:
        lines = f.readlines()
        for l in lines:
            if l != "\n":
                data =  list(eval(l))
                data.sort(key=lambda tup: tup[1], reverse=True)
                pred[method].append([i[0].replace(' ', '') for i in data if i[1] > 0])

In [3]:
df = pd.read_csv('instances_with_human_rationales/class_1_instances.csv')

In [4]:
pred = {}

for method in ['lime', 'shap']:
    pred[method] = []
    read_files('explanations_SHAP&LIME/results-class-1-'+method+'.txt', method, 1)

In [5]:
truth = []

for line in df['Rationales']:
    l = line.split(' ')
    for i, e in enumerate(l):
        l[i] = e.replace(';', '').replace(',', '')
    truth.append(l)

In [6]:
def partial_match_score(pred, truth, method,threshold=0.5):
    ious = []
    
    for a, b in zip(pred[method], truth):
        
        intersection= list(set(a) & set(b))
        union = list(set().union(a, b))
        ious.append(len(intersection)/len(union))

  
    threshold_tps = sum(int(x >= threshold) for x in ious)
    
    precision = threshold_tps/len(pred[method])
    recall = threshold_tps/len(truth)
    f1_score = (2*((recall*precision)/(recall+precision)))
    
    return f1_score

In [7]:
def score_hard_rationale_predictions(pred, truth, method):
    f1_score = []
    recs = []
    precs = []
    for a, b in zip(pred[method], truth):
        
        precision = len(list(set(a) & set(b)))/len(a) if len(a) > 0 else 0
        precs.append(precision)
        recall = len(list(set(a) & set(b)))/len(b) if len(b) > 0 else 0
        recs.append(recall)
        
        if recall+precision == 0:
            f1_score.append(0)
        else:
            f1_score.append(2*((recall*precision)/(recall+precision)))
    
           
    return np.mean(f1_score), np.mean(precs), np.mean(recs)

In [8]:
results = {}
results['Method'] = ['Lime', 'SHAP']
results['IOU_F1'] = []
results['IOU_F1'].append(partial_match_score(pred, truth, 'lime'))
results['IOU_F1'].append(partial_match_score(pred, truth, 'shap'))

results['Token-level Precision'] = []
results['Token-level Recall'] = []
results['Token-level F1'] = []

f1, prec, rec = score_hard_rationale_predictions(pred, truth, 'lime')
results['Token-level Precision'].append(prec)
results['Token-level Recall'].append(rec)
results['Token-level F1'].append(f1)
f1, prec, rec = score_hard_rationale_predictions(pred, truth, 'shap')
results['Token-level Precision'].append(prec)
results['Token-level Recall'].append(rec)
results['Token-level F1'].append(f1)

In [9]:
pd.DataFrame(results)

,Method,IOU_F1,Token-level Precision,Token-level Recall,Token-level F1
0,Lime,0.109804,0.437860,0.391349,0.369832
1,SHAP,0.152941,0.431247,0.511141,0.428473
